In [ ]:
import rasterio
import numpy as np
import xarray as xr
import hvplot.xarray
import holoviews as hv
import geoviews as gv
#import rioxarray as rx
import os
import panel as pn
from holoviews import opts
hv.extension('bokeh', width=90)
pn.extension()
import xrspatial as xrs
from datashader.colors import Elevation
from datashader.transfer_functions import shade

In [ ]:
os.environ["AWS_REQUEST_PAYER"] = "requester" 

# Basic idea with map and max and min height

In [ ]:
places={}
places['St.Thomas']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif"
places['St.Croix']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n18w065/USGS_1_n18w065.tif"
places['Cornell']='https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n42w076/USGS_1_n42w076.tif'

In [ ]:
data = xr.open_rasterio(places['St.Thomas'])

In [ ]:
maxheight = float(data.where(data>0).max())
minheight = float(data.where(data>0).min())

In [ ]:
css = '''
.panel-widget-box {
  background: #f0f0f0;
  border-radius: 5px;
  border: 1px black solid;
}
'''
pn.extension(raw_css=[css])

In [ ]:
title = pn.pane.Markdown('# Visualizing Graphical Data')
greatest = pn.pane.Markdown('# Max height is '+ str(maxheight))
least = pn.pane.Markdown('# The Min height is '+ str(minheight))

In [ ]:
def options(Where):
    data = xr.open_rasterio(Where)
    data = data.where(data>0)[0,:,:]
    maxheight = round(float(data.where(data>0).max()))
    minheight = round(float(data.where(data>0).min()))
    greatest.object = '# Max height: '+ str(maxheight)
    least.object = '# Min height: '+ str(minheight)
    return gv.tile_sources.OSM*data.hvplot.quadmesh(rasterize=True,geo=True,colormap='viridis',project = True, tiles = 'OSM')

In [ ]:
pn.Column(
    title,
    pn.Row(
        pn.Column(pn.interact(options, Where=places),css_classes=['panel-widget-box'],margin=25),
        pn.Column(greatest,least,css_classes=['panel-widget-box'],margin=25),
        margin = (50,100),
        background='#e3c3ee',
        css_classes=['panel-widget-box']
    ),
    margin = 100,
    css_classes=['panel-widget-box'],
    background='#f96080'
)

# Try to bring in xarray-spatial

In [ ]:
hvplot.help('image', docstring=False, generic=False)

In [ ]:
forxr = data.where(data>0)[0,:,:]

In [ ]:
hillshaded = xrs.hillshade(forxr)

In [ ]:
slates = xrs.quantile(forxr,k=100,ignore_vals=(0,))

In [ ]:
#hillshaded.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = 'kg')
slates.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = 'inferno')

# Putting it all together

In [ ]:
maps = ['inferno','blues','kb','kgy','viridis']
choices = ['Shaded','Quantile','Height Range','Slope','Aspect']

In [ ]:
def represent(Location = places['St.Thomas'], DisplayType = 'Shaded', ColorMap='inferno'):
    data = xr.open_rasterio(Location)
    data = data.where(data>0)[0,:,:]
    maxheight = round(float(data.where(data>0).max()))
    minheight = round(float(data.where(data>0).min()))
    greatest.object = '# Max height: '+ str(maxheight)
    least.object = '# Min height: '+ str(minheight)
    if DisplayType == 'Shaded':
        shaded = xrs.hillshade(data)
        return shaded.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = ColorMap, alpha = 20)
    elif DisplayType == 'Quantile':
        disp = lambda level: xrs.quantile(data,k=level,ignore_vals=(0,)).hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = ColorMap)
        graph =pn.interact(disp, level = pn.widgets.IntSlider(start=1,end=50,step=1,value=10))
        return pn.Row(graph)
    elif DisplayType == 'Height Range':
        maxheight = round(float(data.where(data>0).max()))
        minheight = round(float(data.where(data>0).min()))
        slider=pn.widgets.RangeSlider(start=minheight,end=maxheight,step=1,value=(minheight,maxheight))
        hrange = lambda border: xrs.hillshade(data.where(np.logical_and(data>border[0],data<border[1]))).hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = ColorMap)
        graph =pn.interact(hrange, border = slider)
        return pn.Row(graph)
    elif DisplayType == 'Slope':
        sloped = xrs.slope(data)
        maxslope = round(float(sloped.where(sloped>0).max()))
        minslope = round(float(sloped.where(sloped>0).min()))
        slider=pn.widgets.RangeSlider(start=minslope,end=maxslope,step=1,value=(minslope,maxslope))
        srange = lambda border: sloped.where(np.logical_and(sloped>border[0],sloped<border[1])).hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = ColorMap)
        graph =pn.interact(srange, border = slider)
        return pn.Row(graph)
    elif DisplayType == 'Aspect':
        aspect = xrs.aspect(data)
        maxasp = round(float(aspect.where(aspect>0).max()))
        minasp = round(float(aspect.where(aspect>0).min()))
        slider=pn.widgets.RangeSlider(start=minasp,end=maxasp,step=1,value=(minasp,maxasp))
        arange = lambda border: aspect.where(np.logical_and(aspect>border[0],aspect<border[1])).hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = ColorMap)
        graph =pn.interact(arange, border = slider)
        return pn.Row(graph)

In [ ]:
maxheight = round(float(data.where(data>0).max()))
minheight = round(float(data.where(data>0).min()))

In [ ]:
greatest = pn.pane.Markdown('# Max height is '+ str(maxheight))
least = pn.pane.Markdown('# The Min height is '+ str(minheight))

In [ ]:
dashboard = pn.Row(pn.interact(represent, Location = places, DisplayType = choices, ColorMap = maps),
       pn.Column(greatest,least,css_classes=['panel-widget-box'],margin=25))

In [ ]:
#dashboard

In [ ]:
dashboard.servable()